In [28]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

def solve():
    input = sys.stdin.readline

    T = int(input())
    for _ in range(T):

        K = int(input())
        files = list(map(int, input().split()))

        S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
        for i in range(1, K + 1):
            S[i] = S[i-1] + files[i - 1]

        # dp[i][j] i부터 j 까지 결합하는데 최소 시간
        dp = [[0] * (K + 1) for _ in range(K + 1)]

        for length in range(2, K + 1):
            for i in range(1, K - length + 2):
                j = i + length - 1
                candi = [dp[i][k] + dp[k+1][j] for k in range(i, j)]

                # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
                dp[i][j] = min(candi) + (S[j] - S[i-1])

        print(dp[1][K])
solve()

300
864


In [35]:
# 1	11066	 파일 합치기
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
40 30 30 50
15
1 21 3 4 5 35 5 4 3 5 98 21 14 17 32''')

input = sys.stdin.readline

def solve():
    K = int(input())
    files = list(map(int, input().split()))

    S = [0] * (K + 1)  # files 0번 부터 idx 개의 챕터의 누적합
    for i in range(1, K + 1):
        S[i] = S[i-1] + files[i - 1]

    # dp[i][j] i부터 j 까지 결합하는데 최소 시간
    dp = [[0] * (K + 1) for _ in range(K + 1)]
    min_k = [[0] * (K + 1) for _ in range(K + 1)]

    for i in range(1, K):
        dp[i][i+1] = files[i-1] + files[i]
        min_k[i][i+1] = i # 분기점 저장

    for length in range(3, K + 1):
        for i in range(1, K - length + 2):
            j = i + length - 1

            start_k = min_k[i][j-1]
            end_k = min_k[i+1][j]

            min_val = float('inf')
            opt_k = -1

            # 크누스 최적화 적용 (증명됨) 
            for k in range(start_k, end_k + 1):
                val = dp[i][k] + dp[k+1][j]
                if val < min_val:
                    min_val = val
                    opt_k = k
            
            # 구간을 두 부분으로 나누어 합이 최소가 되는 값 + 구간의 파일 크기 
            dp[i][j] = min_val + (S[j] - S[i-1])
            min_k[i][j] = opt_k

    print(dp[1][K])

T = int(input())
for _ in range(T):
    solve()

300
864


In [ ]:
# 2	11049	 행렬 곱셈 순서
import sys

from io import StringIO
sys.stdin = StringIO('''3
5 3
3 2
2 6''')

input = sys.stdin.readline

def solve():

    N = int(input())
    mats = [(0, 0)]
    for _ in range(N):
        mats.append(tuple(map(int, input().split())))

    dp = [[0] * (N + 1) for _ in range(N + 1)]

    for length in range(2, N + 1):
        for start in range(1, N - length + 2):
            end = start + length - 1

            row_start = mats[start][0]
            col_end = mats[end][1]
            min_multi_val = float('inf')

            dp_start = dp[start]
            for mid in range(start, end):
                lv = dp_start[mid]
                rv = dp[mid + 1][end]
                new = row_start * mats[mid][1] * col_end
                candi_val = lv + rv + new

                if candi_val < min_multi_val:
                    min_multi_val = candi_val

            dp[start][end] = min_multi_val

    print(dp[1][N])


solve()

90


In [53]:
# 3	1520	 내리막 길
import sys
sys.setrecursionlimit(10**9)

from io import StringIO
sys.stdin = StringIO('''4 5
50 45 37 32 30
35 50 40 20 25
30 30 25 17 28
27 24 22 15 10''')

input = sys.stdin.readline

def solve():
    M, N = map(int, input().split())
    mountain = [list(map(int, input().split())) for _ in range(M)]

    directions = [
        (+1, 0), (0, +1),
        (-1, 0), (0, -1)
    ]

    # dp[i][j] = ij 에서 M, N 까지 도달 할 수 있는 경우의 수 

    dp = [[-1] * N for _ in range(M)]

    def dfs(r, c):

        if r == M-1 and c == N-1:
            return 1
        
        if dp[r][c] != -1:
            return dp[r][c]
        
        ways = 0
        for dr, dc in directions:
            nr, nc = r + dr, c + dc

            if 0 <= nr < M and 0 <= nc < N:
                if mountain[r][c] > mountain[nr][nc]:
                    ways += dfs(nr, nc)
        
        dp[r][c] = ways
        return ways
        
    print(dfs(0, 0))
solve()


3


In [58]:
# 3	1520	 내리막 길
import sys
from heapq import heappop, heappush

from io import StringIO
sys.stdin = StringIO('''4 5
50 45 37 32 30
35 50 40 20 25
30 30 25 17 28
27 24 22 15 10''')

input = sys.stdin.readline


def solve():
    M, N = map(int, input().split())
    mountain = [list(map(int, input().split())) for _ in range(M)]

    directions = [
        (+1, 0), (0, +1),
        (-1, 0), (0, -1)
    ]

    # dp[r][c] : (0, 0) 출발 (r, c) 까지 도달하는 경로 수
    dp = [[0] * N for _ in range(M)]
    dp[0][0] = 1

    pq = [(-mountain[0][0], 0, 0)]  # (-마이너스 높이, row, col)

    while pq:
        h, r, c = heappop(pq)
        height = -h

        if r == M-1 and c == N-1:
            continue

        for dr, dc in directions:
            nr, nc = r + dr, c + dc

            if 0 <= nr < M and 0 <= nc < N:
                nxt_h = mountain[nr][nc]
                if height > nxt_h:
                    if dp[nr][nc] == 0:
                        heappush(pq, (-nxt_h, nr, nc))
                    dp[nr][nc] += dp[r][c]
    print(dp[M-1][N-1])
solve()

3


In [ ]:
# 4	2629	 양팔저울
import sys

from io import StringIO
sys.stdin = StringIO('''4
2 3 3 3
3
1 4 10''')

input = sys.stdin.readline

def solve():

    N = int(input())
    weights = list(map(int, input().split()))

    B = int(input())
    marbles = list(map(int, input().split()))

    possible = {0}

    for w in weights:
        new_sums = set()

        for p in possible:
            new_sums.add(p + w)
            new_sums.add(abs(p-w))

        possible.update(new_sums)

    out = ["Y" if marble in possible else "N" for marble in marbles]
    print(*out)

solve()

Y Y N


In [87]:
# 4	2629	 양팔저울
import sys

from io import StringIO
sys.stdin = StringIO('''4
2 3 3 3
3
1 4 10''')

input = sys.stdin.readline

def solve():

    N = int(input())
    weights = list(map(int, input().split()))

    M = int(input())
    marbles = list(map(int, input().split()))

    # dp[i][j] i 번째 추까지 고려했을때 j 무게 측정 가능 여부
    MAX_WEIGHT = 15000

    dp = [ [False] * (MAX_WEIGHT + 1) for _ in range(N + 1) ]
    dp[0][0] = True

    for i in range(N):
        weight = weights[i]
        trying = dp[i]
        nxt_try = dp[i + 1]
        for j in range(MAX_WEIGHT + 1):
            if trying[j]:
                if j + weight <= MAX_WEIGHT:
                    nxt_try[j + weight] = True
                nxt_try[j] = True
                nxt_try[abs(j - weight)] = True
                
    result = []

    for m in marbles:
        if m > MAX_WEIGHT:
            result.append("N")
        elif dp[N][m]:
            result.append("Y")
        else:
            result.append("N")
    print(*result)

solve()

Y Y N


In [ ]:
# 4	2629	 양팔저울
import sys

from io import StringIO
sys.stdin = StringIO('''4
2 3 3 3
3
1 4 10''')

input = sys.stdin.readline

def solve():

    N = int(input())
    weights = list(map(int, input().split()))

    M = int(input())
    marbles = list(map(int, input().split()))

    # visited[idx][weight] = idx 추 까지 고려했을 때 가능한 무게.

    MAX_WEIGHT = 500 * 30
    visited = [[False] * (MAX_WEIGHT + 1) for _ in range(N + 1)]
    

    def dfs(idx, cur_weight):
        step = visited[idx]
        if not step[cur_weight]:
            step[cur_weight] = True

            if idx == N:
                return 
            
            weight = weights[idx]
            dfs(idx + 1, cur_weight + weight)
            dfs(idx + 1, abs(cur_weight - weight))
            dfs(idx + 1, cur_weight)

    dfs(0, 0)
    
    result = []
    for marble in marbles:
        if marble > MAX_WEIGHT:
            result.append("N")
        elif visited[N][marble]:
            result.append("Y")
        else:
            result.append("N")
    print(*result)
solve()

Y Y N


In [ ]:
# 5	2293	 동전 1
import sys

from io import StringIO
sys.stdin = StringIO('''3 10
1
2
5''')

input = sys.stdin.readline


def solve():

    n, k = map(int, input().split())
    coins = [int(input()) for _ in range(n)]

    dp = [0] * (k + 1)  # dp[i] -> i 원을 만드는 경우의 수
    dp[0] = 1

    for coin in coins:
        for i in range(coin, k + 1):
            dp[i] += dp[i - coin]
    print(dp[k])


solve()

10


In [ ]:
# 6	7579	 앱
import sys

from io import StringIO
sys.stdin = StringIO('''5 60
30 10 20 35 40
3 0 3 5 4''')

input = sys.stdin.readline


def solve():

    N, M = map(int, input().split())
    in_use = list(map(int, input().split()))
    costs = list(map(int, input().split()))

    max_cost = 100*100
    # dp[i][cost] = max_memory (i 번째 인덱스 프로그램까지 고려했을 때  cost 만큼 아낄때 최대 확모 메모리)
    dp = [[-1] * (max_cost + 1) for _ in range(N + 1)]
    dp[0][0] = 0

    for i in range(N):
        nxt_memory = in_use[i]
        nxt_cost = costs[i]

        before = dp[i]
        after = dp[i + 1]

        # 이전 값 그대로
        for befor_cost in range(max_cost + 1):
            after[befor_cost] = before[befor_cost]

        for befor_cost in range(max_cost + 1):
            before_max_memory = before[befor_cost]
            if before_max_memory != -1:

                # 고려하는 프로그램 추가
                after_cost = befor_cost + nxt_cost
                if after_cost <= max_cost:
                    candi_max_memory = after[after_cost]
                    after_max_memory = before_max_memory + nxt_memory
                    after[after_cost] = max(candi_max_memory, after_max_memory)

    for cost, memory in enumerate(dp[N]):
        if memory >= M:
            print(cost)
            return

solve()

6


In [ ]:
# 6	7579	 앱
import sys

from io import StringIO
sys.stdin = StringIO('''5 60
30 10 20 35 40
3 0 3 5 4''')

input = sys.stdin.readline


def solve():

    N, M = map(int, input().split())
    in_use = list(map(int, input().split()))
    costs = list(map(int, input().split()))

    max_cost = sum(costs)

    dp = [0] * (max_cost + 1)

    for i in range(N):
        cur_memory = in_use[i]
        cur_cost = costs[i]

        for j in range(max_cost, cur_cost - 1, -1):
            dp[j] = max(dp[j], dp[j - cur_cost] + cur_memory)

    for cost in range(max_cost + 1):
        if dp[cost] >= M:
            print(cost)
            return

solve()

6


In [ ]:
# 6	7579	 앱
import sys

from io import StringIO
sys.stdin = StringIO('''5 60
30 10 20 35 40
3 0 3 5 4''')

input = sys.stdin.readline


def solve():

    N, M = map(int, input().split())
    in_use = list(map(int, input().split()))
    costs = list(map(int, input().split()))

    dp = {0:0} #{비용 : 확보메모리}

    for i in range(N):
        cur_memory = in_use[i]
        cur_cost = costs[i]

        for cost, memory in list(dp.items()):
            new_cost = cost + cur_cost
            new_memory = memory + cur_memory

            if new_cost not in dp or dp[new_cost] < new_memory:
                dp[new_cost] = new_memory

    min_cost = float('inf')
    for cost, memory in dp.items():
        if memory >= M and cost < min_cost:
                min_cost = cost
    print(min_cost)
solve()

6


In [ ]:
# 7	11062	 카드 게임
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
1 2 5 2
9
1 1 1 1 2 2 2 2 2''')

input = sys.stdin.readline

def solve():

    n = int(input())
    cards = list(map(int, input().split()))

    # dp[i][j]  = i, j 구간에서 근우가 얻을 수 있는 최대 점수.
    dp = [ [0] * (n) for _ in range(n) ]

    # 1개 남았을때 누구 턴인가? 
    # 홀수로 시작하면 마지막 카드는 근우
    # 짝수로 시작하면 마지막 카드는 명우

    turn = (n - 1) % 2 == 0
    # turn 이 True면, 마지막 카드는 근우

    if turn: 
        for i in range(n):
            dp[i][i] = cards[i]

    # lenth = 남은카드 길이
    for length in range(2, n + 1):
        turn = not turn
        for i in range(n - length + 1):
            j = i + length - 1
            if turn:
                dp[i][j] = max(cards[i] + dp[i+1][j], dp[i][j-1] + cards[j])
            else:
                dp[i][j] = min(dp[i+1][j], dp[i][j-1])

    print(dp[0][n-1])

T = int(input())
for _ in range(T):
    solve()

6
  0   2   2   6
  0   0   5   5
  0   0   0   5
  0   0   0   0

8
  1   1   2   2   4   4   6   6   8
  0   1   1   2   2   4   4   6   6
  0   0   1   1   3   3   5   5   7
  0   0   0   1   1   3   3   5   5
  0   0   0   0   2   2   4   4   6
  0   0   0   0   0   2   2   4   4
  0   0   0   0   0   0   2   2   4
  0   0   0   0   0   0   0   2   2
  0   0   0   0   0   0   0   0   2



In [97]:
# 7	11062	 카드 게임
import sys
sys.setrecursionlimit(2000)

from io import StringIO
sys.stdin = StringIO('''2
4
1 2 5 2
9
1 1 1 1 2 2 2 2 2''')

input = sys.stdin.readline

def solve():

    n = int(input())
    cards = list(map(int, input().split()))

    # dp[i][j]  = i, j 부터 카드가 남았을 때 근우가 얻을 수 있는 최대 점수.
    dp = [ [-1] * (n) for _ in range(n) ]

    def game(turn, i, j):

        if i > j: return 0

        if dp[i][j] != -1:
            return dp[i][j]
        
        # 근우턴
        if turn:
            pick_left = cards[i] + game(False, i + 1, j)
            pick_right = cards[j] + game(False, i, j -1)

            dp[i][j] = max(pick_left, pick_right)

        else:
            pick_left = game(True, i + 1, j)
            pick_right = game(True, i, j - 1)

            dp[i][j] = min(pick_left, pick_right)
        
        return dp[i][j]
    print(game(True, 0, n-1))

T = int(input())
for _ in range(T):
    solve()

6
8


In [ ]:
# 7	11062	 카드 게임
import sys

from io import StringIO
sys.stdin = StringIO('''2
4
1 2 5 2
9
1 1 1 1 2 2 2 2 2''')

input = sys.stdin.readline

def solve():

    n = int(input())
    cards = list(map(int, input().split()))

    prefix = [0] * (n + 1)

    # dp[i][j]  = i, j 구간에서 플레이어가 얻을 수 있는 최고 점수
    dp = [ [0] * (n) for _ in range(n) ]

    # prifix[i + 1] = 0 부터 i 까지의 합
    for i in range(n):
        prefix[i+1] = prefix[i] + cards[i]
    
    for i in range(n):
        dp[i][i] = cards[i]

    for length in range(2, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            cur_range_sum = prefix[j+1] - prefix[i]
            dp[i][j] = cur_range_sum - min(dp[i+1][j], dp[i][j-1])

    print(dp[0][n-1])

T = int(input())
for _ in range(T):
    solve()

6
8
